# Setup

In [2]:
%matplotlib inline

import numpy as np
import scipy.signal as sig
import scipy.stats as stat
import matplotlib.pyplot as plt
import seaborn as sns
import os
import h5py
import resin
import pandas as pd
#import bark  # need to load Bird 4LL data

from pandas import DataFrame,Series,read_table


General info

In [ ]:
savePlots = False    # whether or not to save plots
saveData = False     # whether or not to save csv files

saveAsPath = './Epoch-by-epoch_variables/'
if not os.path.exists(saveAsPath):
    os.mkdir(saveAsPath)
saveAsName = ''

In [ ]:

birdPaths =    ['../data_copies/01_PreprocessedData/01_BudgieFemale_green1/00_Baseline_night/',
                '../data_copies/01_PreprocessedData/02_BudgieMale_yellow1/00_Baseline_night/',
                '../data_copies/01_PreprocessedData/03_BudgieFemale_white1/00_Baseline_night/',
                '../data_copies/01_PreprocessedData/04_BudgieMale_yellow2/00_Baseline_night/',
                '../data_copies/01_PreprocessedData/05_BudgieFemale_green2/00_Baseline_night/']

arfFilePaths =  ['EEG 2 scored/',
                 'EEG 3 scored/',
                 'EEG 3 scored/',
                 'EEG 4 scored/',
                 'EEG 4 scored/']

### load BEST EEG channels - as determined during manual scoring ####
channelsToLoadEEG_best = [['5 LEEGf-LEEGp', '6 LEEGm-LEEGp'],
                     ['5 LEEGf-LEEGm', '4 LEEGf-Fgr'],
                     ['4LEEGf-LEEGp', '9REEGm-REEGp'],
                     ['9REEGf-REEGp', '6LEEGm-LEEGf'],
                     ['4LEEGf-LEEGp','7REEGf-REEGp']]


### load ALL of EEG channels ####
channelsToLoadEEG = [['4 LEEGf-Fgr', '5 LEEGf-LEEGp', '6 LEEGm-LEEGp', '7 LEEGp-Fgr', '8 REEGp-Fgr','9 REEGp-LEEGp'],
                     ['4 LEEGf-Fgr','5 LEEGf-LEEGm', '6 LEEGm-LEEGp', '7 REEGf-Fgr', '8 REEGm-Fgr', '9 REEGf-REEGm'],
                     ['4LEEGf-LEEGp', '5LEEGf-LEEGm', '6LEEGm-LEEGp', '7REEGf-REEGp', '8REEGf-REEGm', '9REEGm-REEGp'],
                     ['4LEEGf-LEEGp', '5LEEGm-LEEGp', '6LEEGm-LEEGf', '7REEGf-Fgr', '8REEGf-REEGm','9REEGf-REEGp',],
                     ['4LEEGf-LEEGp', '5LEEGf-LEEGm', '6LEEGm-LEEGp', '7REEGf-REEGp', '8REEGf-REEGm', '9REEGm-REEGp']]


channelsToLoadEOG = [['1 LEOG-Fgr', '2 REOG-Fgr'],
                     ['2 LEOG-Fgr', '3 REOG-Fgr'],
                     ['2LEOG-Fgr', '3REOG-Fgr'],
                     ['2LEOG-Fgr', '3REOG-Fgr'],
                     ['2LEOG-Fgr', '3REOG-Fgr']]

birds_LL = [1,2,3]
nBirds_LL = len(birds_LL)

birdPaths_LL = ['../data_copies/01_PreprocessedData/02_BudgieMale_yellow1/01_Constant_light/',
                '../data_copies/01_PreprocessedData/03_BudgieFemale_white1/01_Constant_light/',
                '../data_copies/01_PreprocessedData/04_BudgieMale_yellow2/01_Constant_light/',]

arfFilePaths_LL =  ['EEG 2 preprocessed/',
                    'EEG 2 preprocessed/',
                    'EEG 2 preprocessed/']

lightsOffSec = np.array([7947, 9675, 9861 + 8*3600, 9873, 13467])  # lights off times in seconds from beginning of file
lightsOnSec = np.array([46449, 48168, 48375+ 8*3600, 48381, 52005]) # Bird 3 gets 8 hours added b/c file starts at 8:00 instead of 16:00

epochLength = 3
sr = 200
scalingFactor = (2**15)*0.195       # scaling/conversion factor from amplitude to uV (when recording arf from jrecord)

stages = ['w','d','u','i','s','r'] # wake, drowsy, unihem sleep, intermediate sleep, SWS, REM
stagesSleep =    ['u','i','s','r']

stagesVideo = ['m','q','d','s','u'] # moving wake, quiet wake, drowsy, sleep, unclear

## Path to scores formatted as CSVs
formatted_scores_path = '../formatted_scores/'



In [ ]:
colors = sns.color_palette(np.array([[234,103,99],
[218,142,60],
[174,174,62],
[97,188,101],
[140,133,232],
[225,113,190]])
/255)

sns.palplot(colors)

# colorpalette from iWantHue

Plot-specific info

In [ ]:
sns.set_context("notebook", font_scale=1.5)
sns.set_style("white")

# Markers for legends of EEG scoring colors
legendMarkersEEG = []
for stage in range(len(stages)):
    legendMarkersEEG.append(plt.Line2D([0],[0], color=colors[stage], marker='o', linestyle='', alpha=0.7))  

Calculate general variables

In [ ]:
lightsOffEp = lightsOffSec / epochLength
lightsOnEp = lightsOnSec / epochLength

nBirds = len(birdPaths)

epochLengthPts = epochLength*sr

nStages = len(stagesSleep)

#birds = ['1', '2', '3', '4', '5', '2LL']
birds = ['1', '2', '3', '4', '5', '2LL', '3LL', '4LL'] # import LL

## Read in EEG files

In [ ]:
EEGdataAll = {}
TimeIndexEEG = {}
TimeIndexPower = {}

for b_num in birds: 
    
    b = int(b_num[0]) - 1   
        
##### arf files for all datasets except 4LL
    
    if '4LL' not in b_num:
    
        if 'LL' in b_num:
            arf_path = birdPaths_LL[b-1] + arfFilePaths_LL[b-1]
        else:
            arf_path = birdPaths[b] + arfFilePaths[b]

        for channel in channelsToLoadEEG[b]:
            all_data_array = np.array([])

            for file in np.sort(os.listdir(arf_path)):
                if file.endswith('.arf'):
                    arffile = h5py.File(arf_path+file, 'r')
                    data_array = arffile['.'][channel][()]
                    data_array = np.ndarray.flatten(data_array)

                    # Pad the end with NaN's to make it divisible by epoch length
                    nanPadding = np.zeros(epochLengthPts - np.mod(len(data_array), epochLengthPts))
                    nanPadding.fill(np.nan)
                    data_array = np.append(data_array,nanPadding)   

                    all_data_array = np.append(all_data_array,data_array)

            # Do not reshape

            # Save in dict under bird number and channel
            data_name = 'Bird ' + b_num + ': ' + channel
            EEGdataAll[data_name] = scalingFactor * all_data_array
    
##### bark files for 4LL

    elif '4LL' in b_num:
        if 'LL' in b_num:
            arf_path = birdPaths_LL[b-1] + arfFilePaths_LL[b-1]
        else:
            arf_path = birdPaths[b] + arfFilePaths[b]
        
        for channel in channelsToLoadEEG[b]:
            all_data_array = np.array([])

            for file in np.sort(os.listdir(arf_path)):
                if file.endswith('.dat'):
                    dset = bark.read_sampled(arf_path + file)

                    # get channel number
                    ch_num = [x for x in dset.attrs['columns'].keys() if channel in dset.attrs['columns'][x]['channel_name']]
                    if len(ch_num) != 1:
                        print("for bird", b_num, "channel", channel, ": correct number of channels not found")

                    data_array = dset.data[:, ch_num]
                    data_array = np.array(np.ndarray.flatten(data_array))  # flatten and take out of memmap modes

                    # Pad the end with NaN's to make it divisible by epoch length
                    nanPadding = np.zeros(epochLengthPts - np.mod(len(data_array), epochLengthPts))
                    nanPadding.fill(np.nan)
                    data_array = np.append(data_array,nanPadding)   

                    all_data_array = np.append(all_data_array,data_array)

                # Do not reshape

                # Save in dict under bird number and channel
                data_name = 'Bird ' + b_num + ': ' + channel
                EEGdataAll[data_name] = scalingFactor * all_data_array
    
##### Create time index for EEG

    all_time_array = np.array([], dtype='datetime64')
    for file in np.sort(os.listdir(arf_path)):
        if file.endswith('.arf')|file.endswith('.dat'):

            date = file.split('_')[2]

            if (b == 0) & ('2014-10-17' in file):
                hours = '17'
                minutes = '32'
            elif b == 0:
                hours = '09'
                minutes = '00'
            else:
                time = file.split('_')[3].split('.')[0]
                hours = time.split('-')[0]
                minutes = time.split('-')[1]

            datetime_start = np.datetime64(date + 'T' + hours + ':' + minutes + ':06')    # assume 6-s delay in starting recording

            # time index in datetime format
            
            if file.endswith('.arf'):
                arffile = h5py.File(arf_path+file, 'r')
                length_pts   = len(arffile['.'][channel][()])
            elif file.endswith('.dat'):
                length_pts   = len(bark.read_sampled(arf_path + file).data[:,0])
            
            padding_pts = epochLengthPts - np.mod(length_pts, epochLengthPts)
            length_s = (length_pts+padding_pts)/sr
            length_ms = np.timedelta64(int(1000 * length_s), 'ms')
            datetime_end = datetime_start + length_ms

            time_array = np.arange(datetime_start, datetime_end, np.timedelta64(int(1000/sr),'ms')) 

            # Add to end of whole-night time index
            all_time_array = np.append(all_time_array, time_array)


    data_name = 'Bird ' + b_num
    TimeIndexEEG[data_name] = all_time_array
    
    # Get time at the start of each epoch
    TimeIndexPower[data_name] = all_time_array[np.arange(0,len(all_time_array),epochLengthPts)]
    

EEGchannels = np.sort(list(EEGdataAll.keys()))

# Load formatted scores

## Bring scores into alignment with padded EEGs

In [ ]:
AllScores = {}
for b in birds:
    # Load from file
    scores_file = 'All_scores_Bird {}.csv'.format(b)
    tmp_scores = pd.read_table(formatted_scores_path + scores_file, sep=',', index_col=0)

    # Reindex according to TimeIndexPower for that bird
    tmp_time_index = TimeIndexPower['Bird ' + str(b)]
    tmp_scores = tmp_scores.reindex(pd.to_datetime(tmp_scores.index)) # convert original index to pandas datetimes
    tmp_scores = tmp_scores[~tmp_scores.index.duplicated()] # remove any duplicates
    reindexed_scores = tmp_scores.reindex(pd.to_datetime(tmp_time_index)) # reindex with EEG-based index
    
    # save to dict
    AllScores['Bird ' + str(b)] = reindexed_scores    

## Calculate lights off in Zeitgeber time (s and hrs)
Lights on is 0 

In [ ]:
lightsOffDatetime = np.array([], dtype='datetime64')
lightsOnDatetime = np.array([], dtype='datetime64')

for b_num in range(nBirds):
    b_name = 'Bird ' + str(b_num+1)
    Scores = AllScores[b_name]
    startDatetime = np.datetime64(Scores.index.values[0])

    # Calc lights off & on using datetime formats
    lightsOffTimedelta = lightsOffSec[b_num].astype('timedelta64[s]')
    lightsOffDatetime = np.append(lightsOffDatetime, startDatetime + lightsOffTimedelta)
    lightsOnTimedelta = lightsOnSec[b_num].astype('timedelta64[s]')
    lightsOnDatetime = np.append(lightsOnDatetime, startDatetime + lightsOnTimedelta)

In [ ]:
lightsOffZeit_s = lightsOffSec - lightsOnSec
lightsOffZeit_hr = lightsOffZeit_s / 3600

### Mark artifacts

#### Convert score labels to numbers: 
* mark any 'moving' video-labels as -1

In [ ]:
for bird in birds:
    bird = 'Bird ' + b
    Scores = AllScores[bird]
    # replace nan's with empty string
    Scores.fillna('', inplace=True)
    
    if "LL" not in b:

        Label_num = -1 * np.ones_like(Scores['Label'])
        for st in range(len(stages)):
            stage_inds = [x for x in range(len(Scores['Label'])) if stages[st] in Scores['Label'].iloc[x]]
            Label_num[stage_inds] = st

        # Moving labels
        stage_video_inds = [x for x in range(len(Scores['Label'])) if ('m' in Scores['Video Label'].iloc[x])]
        Label_num[stage_video_inds] = -1

        # Save to dataframe
        AllScores[bird]['Label (#)'] = Label_num


#### for each channel, mark as artifact epochs w/ data that crosses an amplitude threshold

# Set thresholds
#artifact_threshold_uV = 400
artifact_threshold_SD  = 6   # of SDs away from mean

# Make a scores array for each channel so it has independent artifact removal
ChannelScores = {}

for ch in EEGchannels:
    data = EEGdataAll[ch]
    # mean + N*SD
    artifact_threshold_SD_uV = (data[~np.isnan(data)]).mean() + artifact_threshold_SD*(data[~np.isnan(data)]).std()
    print(ch + ' : ' + str(artifact_threshold_SD_uV))
    
    #artifact_threshold = np.max([artifact_threshold_uV, artifact_threshold_SD_uV])
    artifact_threshold = artifact_threshold_SD_uV
        
    b_name = ch[0:6]
    bird_scores = np.array(AllScores[b_name]['Label (#)'].values)   # get scores as an array of numbers
    nEpochs = len(bird_scores)

    for ep in range(nEpochs):
        start_pts = ep * epochLengthPts
        stop_pts  = (ep+1) * epochLengthPts
        
        ep_data = data[start_pts:stop_pts]
        
        if any(np.abs(ep_data) > artifact_threshold):
            bird_scores[ep] = -2
            
    # Save to dataframe
    ChannelScores[ch] = bird_scores

### load_EM_artifacts

In [ ]:
def load_EM_artifacts(b):
    birdPath = '0{}_Bird_{}_SW_EM_detected/'.format(b,b)
    path = events_path + birdPath

    all_ch_data = {}   # init

    for file in os.listdir(path):
        if 'EMartifacts' in file:
            # Get channel
            ch_tmp = file.split('_')[-1]
            ch = ch_tmp.split('.')[0]
            ch_data = pd.read_table(path+file, sep=',', index_col=0)

            if ch in all_ch_data.keys():
                all_ch_data[ch] = all_ch_data[ch].append(ch_data)
            else:
                all_ch_data[ch] = ch_data

    channel_names = list(all_ch_data.keys())
    
    return(all_ch_data)

## assign_epoch

In [ ]:
def assign_epoch(data):
    '''data: a single Dataframe (eg, for SWs, must provide all_ch_data[ch])'''

    epochID = np.floor(data['Start'] / epochLength)
    data['Epoch # start'] = epochID
    data['Epoch #'] = epochID
    epochID = np.floor(data['Stop'] / epochLength)
    data['Epoch # stop'] = epochID
    data['Length'] = data['Stop'] - data['Start']
    
    # Detect & split overlaps (ie events spanning more than one epoch)
    overlaps = data[data['Epoch # start'] != data['Epoch # stop']]
    non_overlaps_tmp = data[data['Epoch # start'] == data['Epoch # stop']]
    non_overlaps = non_overlaps_tmp[['Label', 'Epoch #', 'Start', 'Stop', 'Length']]

    # Dataframe for first half of event
    overlaps_first_ep = pd.DataFrame(overlaps[['Label', 'Epoch #', 'Start']])
    overlaps_first_ep['Stop'] = epochLength * overlaps['Epoch # stop'].values
    overlaps_first_ep['Length'] = overlaps_first_ep['Stop'] - overlaps_first_ep['Start']

    # Dataframe for second half of event
    overlaps_second_ep = pd.DataFrame(overlaps[['Label']])
    overlaps_second_ep['Epoch #'] = overlaps['Epoch # stop'].values
    overlaps_second_ep['Start'] = epochLength * overlaps['Epoch # stop'].values
    overlaps_second_ep['Stop'] = overlaps['Stop'].values
    overlaps_second_ep['Length'] = overlaps_second_ep['Stop'] - overlaps_second_ep['Start']

    # Concatenate non-overlaps with the split events
    data = pd.concat([overlaps_first_ep, overlaps_second_ep, non_overlaps], ignore_index=True)
    data = data.sort_values(by='Start')
    data = data.reset_index(drop=True)
    
    return(data)

EMartifacts = {}

for b in birds:

    tmp_bird_EMartifacts = load_EM_artifacts(b)
    channels = tmp_bird_EMartifacts.keys()
    for ch in channels:

        data = assign_epoch(tmp_bird_EMartifacts[ch])
        Artifact_sec_per_epoch = data.groupby(['Epoch #'])['Length'].sum() ## currently using this one
        Artifact_max_duration_per_epoch = data.groupby(['Epoch #'])['Length'].max()

        ch_name = 'Bird ' + str(b) + ': ' + ch
        EMartifacts[ch_name] = Artifact_sec_per_epoch

# For each bird find epochs with too much EM artifact
EM_artifact_thres = .5  # in seconds/epoch
for ch in EEGchannels:

    bird = int(ch[5])-1
    bird_name = "Bird " + ch[5]

    # Get EM artifact data for that channel
    tmp_data = EMartifacts[ch]
    tmp_data = tmp_data.reindex(AllScores[bird_name]['Epoch #'])
    

    too_many_artifacts = tmp_data > EM_artifact_thres

    ch_scores = np.array(ChannelScores[ch])
    inds_artifacts = np.where(too_many_artifacts)[0]
    print(ch + ": " + str(len(inds_artifacts)) + " epochs removed")
    print(pd.Series(ch_scores[inds_artifacts]).value_counts())
    ch_scores[inds_artifacts] = -3

    # Save to dataframe
    ChannelScores[ch] = ch_scores

In [ ]:
AllScores['Bird 1']

## Get variables for each 1-s "epoch" of sleep

### Spectral total power

In [ ]:
SleepVariables = {}

# set epoch length for spectral analysis
epochLength_analysis = 1
epochLengthPts_analysis = sr*epochLength_analysis

for ch in EEGchannels:
    
    b_name = ch.split(':')[0]
    # load data
    rawdata = np.ndarray.flatten(EEGdataAll[ch])
    # number of analysis epochs
    nEpochs = int(len(rawdata)/epochLengthPts_analysis)
    
    # initialize
    ch_Delta = np.zeros(nEpochs)
    ch_Gamma = np.zeros(nEpochs)
    ch_GammaDeltaRatio = np.zeros(nEpochs)
    ch_Delta.fill(np.nan)
    ch_Gamma.fill(np.nan)
    ch_GammaDeltaRatio.fill(np.nan)

    # 1. Get multitaper power spectrum: 3s epochs with sliding 1-s window
    
    spec = resin.Spectra(rate=sr, 
        NFFT=epochLengthPts,
        data_window=epochLengthPts,
        noverlap=2*epochLengthPts_analysis,  ## 1-s overlap on either side of window
        n_tapers=2,   # 2 tapers not 4
        NW = 3,
        freq_range=(0, 60))
    spec.signal(rawdata)

    epPower = spec.power()[0]
    epFFTfreqs = spec.power()[1]
        
    # 1a. Save delta and gamma bands
    
    ##### EDITED: DELTA SPECTRUM to match Low et al paper (1 - 4 Hz instead of 0.5 - 4 Hz)
    indsDelta = np.where((epFFTfreqs >= 1)   & (epFFTfreqs <= 4))
    indsGamma = np.where((epFFTfreqs >= 30)  & (epFFTfreqs <= 55))
    
    ch_Delta = np.sum(epPower[indsDelta], axis=0)
    ch_Gamma = np.sum(epPower[indsGamma], axis=0)
    ch_GammaDeltaRatio = ch_Gamma/ch_Delta

    
    # 2a. Log of delta
    ch_DeltaLog = np.log10(ch_Delta)
        
    # 3. Get sleep scores: each 1s epoch gets the score of the entire 3-s epoch it's in
    ### USE ORIGINAL SCORES rather than artifact-removed scores for each channel
    bird_video_scores = np.repeat(AllScores[b_name]['Video Label'].values, 3, axis=0)
    
    #
    if 'LL' in ch:
        is_sleep = bird_video_scores == 's'
    else: ## only have PSG scores for LD datasets
        bird_scores = np.repeat(AllScores[b_name]['Label (#)'].values, 3, axis=0)
        is_sleep = bird_scores > 2
    
    # epoch ID
    epochs_3s = np.repeat(AllScores[b_name]['Epoch #'].values, 3, axis=0)
    
    # 4. Save to dataframe
    sleep_variables = DataFrame([])
    sleep_variables["Delta"] = ch_Delta
    sleep_variables["DeltaLog"] = ch_DeltaLog
    sleep_variables["GammaDeltaRatio"] = ch_GammaDeltaRatio
    sleep_variables["video scores"] = bird_video_scores
    sleep_variables["is sleep"] = is_sleep
    sleep_variables["Epoch"] = epochs_3s[0:len(bird_video_scores)]
    
    if 'LL' not in ch:
        sleep_variables["sleep scores"] = bird_scores

    SleepVariables[ch] = sleep_variables
    

### Spectral gradients

In [ ]:
#####################################################
for ch in EEGchannels:
    
    b_name = ch.split(':')[0]
    # load data
    rawdata = np.ndarray.flatten(EEGdataAll[ch])
    # number of analysis epochs
    nEpochs = int(len(rawdata)/epochLengthPts_analysis)
    
    sleep_variables = SleepVariables[ch].copy()
    
    # 1. Calculate gradients of delta & gamma/delta
                #ch_gradientDelta = np.gradient(ch_Delta)
                #ch_gradientDeltaLog = np.gradient(ch_DeltaLog)
                #ch_gradientGammaDelta = np.gradient(ch_GammaDeltaRatio)
    
    # 2. Drop non-sleep epochs
    sleep_variables = sleep_variables[sleep_variables['is sleep']]
    
    ch_Delta = sleep_variables['Delta']
    ch_DeltaLog = sleep_variables['DeltaLog']
    ch_GammaDeltaRatio = sleep_variables['GammaDeltaRatio']
    
    # 3.  Get time points
    dt = sleep_variables.index.values
    
    # 4.  Calculate gradients of delta & gamma/delta
    ch_gradientDelta = np.gradient(ch_Delta, dt)
    ch_gradientDeltaLog = np.gradient(ch_DeltaLog, dt)
    ch_gradientGammaDelta = np.gradient(ch_GammaDeltaRatio, dt)
    
    
    # 5. Save to dataframe
    sleep_variables["diffDelta"] = ch_gradientDelta
    sleep_variables["diffDeltaLog"] = ch_gradientDeltaLog
    sleep_variables["diffGammaDeltaRatio"] = ch_gradientGammaDelta
    
    SleepVariables[ch] = sleep_variables
    

### SDs, nPeaks, and max amplitude

In [ ]:
#####################################################

for ch in EEGchannels:

    b_name = ch.split(':')[0]
    # load data
    rawdata = np.ndarray.flatten(EEGdataAll[ch])
    # number of analysis epochs
    nEpochs = int(len(rawdata)/epochLengthPts_analysis)
    
    sleep_variables = SleepVariables[ch].copy()

    # 1. Reshape data so 1 row = 1 epoch ####
    all_data_array = np.reshape(rawdata, (nEpochs, epochLengthPts_analysis))
    
    # 2. Init MAX AMPLITUDE PER 3-s EPOCH
    nSleepEpochs = len(sleep_variables)
    ch_max_amp = np.zeros(nEpochs)
    ch_max_amp.fill(np.nan)

    # 3. Init nPeaks per 3-s epoch
    
    ch_nPeaks = np.zeros(nEpochs)
    ch_nPeaks.fill(np.nan)
    
    # 4. Init SD of waveform FOR 3-S EPOCH --- including wake etc
    ch_waveformSD = np.zeros(nEpochs)
    ch_waveformSD.fill(np.nan)

    for ep in sleep_variables.index:
        #### Construct the 3-s windows sliding by 1s
        
        # first epoch
        if ep == 0:
            tmp_3s_window = np.concatenate([all_data_array[ep], all_data_array[ep+1]])

        # last epoch
        elif ep == nEpochs-1:
            tmp_3s_window = np.concatenate([all_data_array[ep-1], all_data_array[ep]])

        # normal epochs
        else:
            tmp_3s_window = np.concatenate([all_data_array[ep-1], all_data_array[ep], all_data_array[ep+1]])

        #### Calculate per-epoch variables
            
        # max amp
        ch_max_amp[ep] = np.max(np.abs(tmp_3s_window))
            
        # nPeaks
        peaks = sig.argrelmax(tmp_3s_window)
        ch_nPeaks[ep] = len(peaks[0])
            
        # SD
        ch_waveformSD[ep] = np.std(tmp_3s_window)
        
                
    # 5. take only sleep epochs
    ch_max_amp = ch_max_amp[sleep_variables.index]
    ch_nPeaks = ch_nPeaks[sleep_variables.index]
    ch_waveformSD = ch_waveformSD[sleep_variables.index]
        
    # don't z-score variables yet, because this will include wake episodes with lots of artifact
    
    # 6. Save to dataframe
    sleep_variables["max amp"] = ch_max_amp
    sleep_variables["SD"] = ch_waveformSD
    sleep_variables["nPeaks"] = ch_nPeaks

    
    SleepVariables[ch] = sleep_variables

In [ ]:
print('done')

### save to csv

In [ ]:
for channel in SleepVariables.keys():
    data = SleepVariables[channel]
    data.to_csv(saveAsPath + channel + '.csv')